<a href="https://colab.research.google.com/github/PlaZMaD/climate/blob/main/file_handling_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключаем ВАШ гугл-диск, как папку. В примере у меня в корне моего диска есть папка weather_data, где лежит файл boloto_2016.xlsx

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
import os
# drive.mount('/content/drive')

Это наши функции, которые мы будем применять.

In [ ]:
import pandas as pd
import numpy as np


def hampel_filter_pandas(input_series, window_size, n_sigmas=3):
    k = 1.4826  # scale factor for Gaussian distribution
    new_series = input_series.copy()
    indices = None
    # # helper lambda function
    MAD = lambda x: np.median(np.abs(x - np.median(x)))

    rolling_median = input_series.rolling(window=2 * window_size, center=True).median()
    rolling_mad = k * input_series.rolling(window=2 * window_size, center=True).apply(MAD)
    diff = np.abs(input_series - rolling_median)
    print(len(diff.index), len(rolling_mad.index))

    indices = diff > (n_sigmas * rolling_mad)  # list(np.argwhere(diff > (n_sigmas * rolling_mad)).flatten())
    # #new_series[indices] = rolling_median[indices]

    return new_series, indices[indices == True].index

def mad_filter(input_df, target_col, z=5.5, fill_method='new'):
    data_if = input_df[[target_col]].copy()
    data_if['rolling_fill'] = data_if[target_col].rolling(10).mean()
    data_if['plus_shift'] = data_if[target_col].shift(1)
    null_index_plus = data_if['plus_shift'].isnull()
    null_index_plus = null_index_plus[null_index_plus == True].index

    data_if['minus_shift'] = data_if[target_col].shift(-1)
    null_index_minus = data_if['minus_shift'].isnull()
    null_index_minus = null_index_minus[null_index_minus == True].index

    if fill_method == 'old':
        data_if.loc[null_index_plus, 'plus_shift'] = data_if.loc[null_index_plus, target_col]
        data_if.loc[null_index_minus, 'minus_shift'] = data_if.loc[null_index_minus, target_col]
    else:
        data_if.loc[null_index_plus, 'plus_shift'] = data_if.loc[null_index_plus, 'rolling_fill']
        data_if.loc[null_index_minus, 'minus_shift'] = data_if.loc[null_index_minus, 'rolling_fill']

    data_if['d_i'] = (data_if[target_col] - data_if['minus_shift']) - (data_if['plus_shift'] - data_if[target_col])

    # print(key, pd.isna(data['d_i']).value_counts())
    d_median = np.median(data_if.query('not (d_i != d_i)')['d_i'])
    MAD = np.median(np.abs(data_if.query('not (d_i != d_i)')['d_i'] - d_median))

    down_threshold = d_median - (z * MAD / 0.6745)
    up_threshold = d_median + (z * MAD / 0.6745)

    out_data = np.logical_or(data_if['d_i'] < down_threshold, data_if['d_i'] > up_threshold, data_if['d_i'].isna())
    return out_data

Проверяем, что наша папка существует, как и файл в ней.

In [ ]:
# !ls /content/drive/MyDrive/weather_data

In [ ]:
!gdown 1uCL0X9sdmMVz4Fyw2mohlNCsQAjifvDy

Указываем как называется колонка, с которой будем работать и путь до файла

In [ ]:
target = 'NEE'#'co2_flux_full output'
# fileName = 'weather_data/for_filtering.xlsx'
fileName = for_filtering.xlsx

Загружаем наш файл, если в нем несколько листов - сливаем в один.

In [ ]:
all_data = pd.read_excel(os.path.join("/content/drive/MyDrive/", fileName),
                         sheet_name=None,          # загружаем все листы! Если надо конкретные, можно их прописать тут
                         skiprows=None)#lambda x: x==1)  # Важный момент! В моем файле вторая строка идет без данных, поэтому я ее вот таким образом пропускаю, уберите эту опцию, если у вас все идет подряд.
data = pd.concat(all_data.values(), ignore_index=True)

In [ ]:
print(data.head())

Тут самое сложное. Надо нормально прочитать временнЫе метки, а они у всех файлов что мне показывали -  разные. В итоге нужна колонка, в которой как-то будут указаны целиком дата и время. Если она есть - отлично, если нет - формируем сами из нескольких имеющихся.

In [ ]:
#data['dateTime'] = data['dateTime'].astype(str)
# data = data.rename(columns={'Unnamed: 0': "dateTime"})
# data['dateTime'] = pd.to_datetime(data['dateTime'],
#                                   format='%d.%m.%Y %H:%M:%S') #это шаблон того, как у нас выглядят наши данные.
# data.index = pd.DatetimeIndex(data['dateTime'])



# data = data.rename(columns={'Unnamed: 0': "dateTime"})
data['dateTime'] = pd.to_datetime(data['timestamp'],
                                  format='%Y-%m-%d %H:%M:%S') #это шаблон того, как у нас выглядят наши данные.
data.index = pd.DatetimeIndex(data['dateTime'])

Помечаем строки с данными.

In [ ]:
data['process_flag'] = np.invert(pd.isnull(data[target]))

Начинаем искать выбросы разными методами...

In [ ]:
data.loc[data['process_flag']==True, 'filter_55'] = mad_filter(data.query('process_flag==True'), target, z=5.5, fill_method='new')
fig = px.scatter(data, x='dateTime', y=target, color='filter_55', title='co2 MAD z=7')
# fig.write_html(f"/content/drive/MyDrive/weather_data/co2_plot.html")
fig.update_yaxes(type="log")
fig.show()

In [ ]:
data.loc[data['process_flag']==True, 'filter_7'] = mad_filter(data.query('process_flag==True'), target, z=7., fill_method='new')
fig = px.scatter(data, x='dateTime', y=target, color='filter_7', title='co2 MAD z=7')
# fig.write_html(f"/content/drive/MyDrive/weather_data/co2_plot.html")
fig.show()

In [ ]:
data['filter_hampel'] = False
rwn, indices = hampel_filter_pandas(data.query('process_flag==True')[target], window_size=10)
data.loc[indices, 'filter_hampel'] = True
fig = px.scatter(data.query('process_flag==True'), x='dateTime', y=target, color='filter_hampel', title='co2 Hampel')
# fig.write_html(f"/content/drive/MyDrive/weather_data/{key}_plot.html")
fig.show()

In [ ]:
data['filter_hampel_OR_55'] = np.logical_or(data['filter_hampel'],data['filter_55'])
fig = px.scatter(data.query('process_flag==True'), x='dateTime', y=target, color='filter_hampel_OR_55', title='co2 or 55')
fig.show()

In [ ]:
data['filter_hampel_after_55'] = False
rwn, indices = hampel_filter_pandas(data.query('process_flag==True & filter_55==False')[target], window_size=10)
data.loc[indices, 'filter_hampel_after_55'] = True
data['filter_hampel_after_55'] = np.logical_or(data['filter_hampel_after_55'], data['filter_55'])
fig = px.scatter(data.query('process_flag==True'), x='dateTime', y=target, color='filter_hampel_after_55', title='co2 Hampel after MAD 5.5')
fig.show()

Если надо - сохраняем наши данные в файл.

In [ ]:
base_dir, out_filename =  os.path.split(os.path.join("/content/drive/MyDrive/", fileName))
out_filename = "out_"+out_filename
data.to_csv(os.path.join(base_dir, out_filename))
